# MBA em Ciência de Dados
# Técnicas Avançadas de Captura e Tratamento de Dados

### <span style="color:darkred">Tratamento e limpeza de Dados</span>


### <span style="color:darkred">Avaliação</span>

Moacir Antonelli Ponti

CeMEAI - ICMC/USP São Carlos

---

In [ ]:
# carregando as bibliotecas necessárias
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re

from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from imblearn import under_sampling

# carregando dados
data = pd.read_csv("./dados/pib_mba_avaliacao.csv")
data

,id,gid,UF,nome,Censo,PIB,Pop_est_2009,PIB_percapita,classe,desemprego,Pop_ativa_est_2009,estimat_pop_desemprego
0,1406,1,RIO GRANDE DO SUL,Igrejinha,2010.0,6.907114e+05,33551,20586.910156,2,5.1,21976,1121
1,4252,2,RIO GRANDE DO SUL,Três Coroas,2010.0,4.102403e+05,24786,16551.289062,2,4.8,17350,833
2,998,3,RIO GRANDE DO SUL,Canoas,2010.0,1.644448e+07,332056,49523.199219,3,4.6,202554,9317
3,4292,4,RIO GRANDE DO SUL,Sapucaia do Sul,2010.0,1.767158e+06,126316,13989.980469,2,4.5,88421,3979
4,991,5,RIO GRANDE DO SUL,São Leopoldo,2010.0,3.302153e+06,211663,15600.990234,2,4.9,138639,6793
...,...,...,...,...,...,...,...,...,...,...,...,...
5562,150,5564,PIAUÍ,Alagoinha do Piauí,2010.0,2.727773e+04,7736,3526.080078,1,4.6,4719,217
5563,4216,5565,PIAUÍ,Olho D'Água do Piauí,2010.0,1.029336e+04,2755,3736.250000,1,6.6,1928,127
5564,137,5566,PIAUÍ,Passagem Franca do Piauí,2010.0,1.441198e+04,4246,3394.250000,1,5.1,2781,142
5565,3861,5567,PIAUÍ,Barro Duro,2010.0,2.505022e+04,6853,3655.370117,1,4.7,4180,196


Vamos utilizar uma base de dados baixada do IBGE com o PIB per capita para cada município brasileiro, essa base foi modificada para o propósito dos exercícios abaixo e possui as seguintes colunas:
* gid - identificador geográfico do município
* UF - unidade federativa
* nome - nome do município
* Censo - ano do censo relativo aos dados. Notar que houve censo agropecuário em 2007 e censo em 2010
* PIB - total do PIB
* Pop_est_2009 - populacao estimada
* PIB_percapita - PIB per capita segundo os dados
* Descrição - Descrição do dados
* classe - classe do município
* desemprego - índice de desemprego na cidade no ano do Censo
* Pop_ativa_est_2009 - populacao ativa estimada
* estimat_pop_desemprego - valor absoluto da população ativa desempregada com base no índice de desemprego



### Questão 1)

Verifique a distribuição dos valores e tipos dos atributos nas colunas: UF, Censo, desemprego, Pop_est_2009 e realize um tratamento de dados considerando:

1. Para campos do tipo string, converta tudo a caixa alta, e não considere essa operação como correção de dados;
2. Para UF, considere como válidos os valores: 'BAHIA', 'RIO GRANDE DO SUL', 'ESPIRITO SANTO', 'MINAS GERAIS', 'PIAUÍ', 'GOIÁS', 'PERNAMBUCO', 'PARÁ', 'SERGIPE', 'SÃO PAULO', 'PARAÍBA', 'TOCANTINS', 'AMAZONAS', 'SANTA CATARINA', 'CEARÁ', 'RORAIMA', 'MARANHÃO', 'RIO DE JANEIRO', 'MATO GROSSO', 'PARANÁ', 'ALAGOAS', 'MATO GROSSO DO SUL', 'ACRE', 'RIO GRANDE DO NORTE', 'RONDÔNIA', 'AMAPÁ', 'DISTRITO FEDERAL';
3. Correção dos dados que for possível inferir o valor verdadeiro, ajustando e padronizando-os. Anote em quais variáveis isso ocorreu e a quantidade de valores corrigidos. Valores outliers não devem ser removidos/alterados, apenas dados não padronizados ou passíveis de correção;
4. Conversão dos atributos cujos valores foram corrigidos no item anterior, e que deveriam ser numéricos, para numérico - inspecione os valores para garantir que a conversão não vá gerar dados faltantes de forma desnecessária, substituindo por numeros os que forem possíveis como por exemplo o atributo "floor" como visto na aula em que substituímos dados por 0. Anote as variáveis em que isso ocorreu.
    
Quais variáveis possuiam valores que precisaram ser padronizados ou corrigidos, e quantos valores em cada uma delas foi possível corrigir?

(a) UF (12) <br>
(b) UF (12) e Pop_est_2009 (4)<br>
(c) UF (12) e Pop_est_2009 (5)<br>
(d) UF (15), Censo (1), Pop_est_2009 (5)<br>
(e) UF (15), Desemprego (1), Pop_est_2009 (4)<br>

---

In [ ]:
data = pd.read_csv("./dados/pib_mba_avaliacao.csv")

-----

### Questão 2)

Recarregue a base de dados.

1. Remova colunas que possuam valores constantes ou redundantes (cujo valor possa ser computado por meio do uso de outras colunas), considere que será necessário identificar os municípios de forma única a partir da coluna `gid`.
2. Exiba, e posteriormente remova muninícipios duplicados (considere apenas as colunas `gid`, `nome`, `UF` e classe para esse fim), mantendo a primeira ocorrência.

Quantas colunas e linhas foram removidas?

(a) 1 coluna, 2 linhas<br>
(b) 2 colunas, 1 linha<br>
(c)  2 colunas, 2 linhas<br>
(d) 3 colunas, 2 linhas<br>
(e) 4 colunas, 2 linhas<br>

In [ ]:
data = pd.read_csv("./dados/pib_mba_avaliacao.csv")

In [ ]:
n_data = data.shape
print("Dados antes tratamento %d x %d" % (n_data[0], n_data[1]))

Dados antes tratamento 5567 x 12


---
### Questão 3)

Vamos analisar possíveis outliers nas colunas numéricas. Utilize o método da análise da dispersão pelo *IQR* e inspecione as colunas 'PIB', 'Pop_est_2009', 'estimat_pop_desemprego' e 'desemprego', procurando por outliers globais para além de 3 IQRs, i.e. valores menores que o quartil 1 menos $IQR \times 3$ ou maiores que o quartil 3 mais $IQR \times 3$. Não remova outliers da base de dados, apenas identifique-os.

Antes de verificar os outliers, converta as colunas Pop_est_2009 e estimat_pop_desemprego para numérico, com opção `downcast='integer', errors='coerce'` para tornar faltantes (nan) os não convertidos corretamente e ignorar esses valores da análise.

Quantos outliers foram encontrados, respectivamente, para 'PIB', 'Pop_est_2009', 'estimat_pop_desemprego' e 'desemprego'?

(a) 382, 233, 220, 1<br>
(b) 405, 273, 338, 0<br>
(c) 405, 373, 338, 5<br>
(d) 506, 373, 449, 0<br>
(e) 506, 373, 338, 1<br>


In [ ]:
data = pd.read_csv("./dados/pib_mba_avaliacao.csv")

---

### Questão 4)

Inicie com os seguintes passos:
1. calcule e imprima a média e a mediana dos valores na coluna 'desemprego', arredondando para 4 casas decimais,
2. converta todos os valores da coluna 'UF' para caixa alta (`upper`),
3. calcule e anote o total de valores faltantes na variável 'desemprego' para cada 'UF' distinta.

Em contato com a equipe especialista, fomos informados que uma boa forma de preencher esses valores seria o preenchimento pela mediana do desemprego no estado referente ao dado faltante. Assim, iremos preencher os valores `nan` de 'desemprego' com as medianas agrupadas por 'UF'.

Para isso codifique uma função que deverá:
1. calcular a mediana de uma variável alvo A (a ser preenchida) relativa a (ou agrupada por) cada valor distinto de uma variável categórica B que se deseja usar para agrupar; no nosso caso A='desemprego' e B='UF';
2. atribuir a mediana calculada de forma agrupada a todas as linhas cuja variável alvo é faltante e que possua o valor da variável categórica correspondente;
3. garantir que o valor atribuido tenha o mesmo tipo da variável alvo, ou seja, int, float, etc. Quando int, realize o arredondamento utlizando `np.round(,0)`, quando float64 utilize `np.round(,2)`

Realize o preenchimento dos dados faltantes utilizando o método da mediana agrupada por UF. A seguir, comute novamente e imprima a média e a mediana dos valores na coluna 'desemprego' agora preenchida, arredondando para 4 casas decimais.

Quantas UFs distintas continham dados faltantes, e qual foi o efeito do preenchimento da variável desemprego na média e na mediana (arredondadas para 4 casas decimais) da variável desemprego (em outras palavras, essas estatísticas reduziram, se mantiveram ou aumentaram após a transformação)?

(a) 5 UFs, a média reduziu, a mediana se manteve<br>
(b) 5 UFs, a média aumentou, a mediana se manteve<br>
(c) 6 UFs, a média reduziu, a mediana reduziu<br>
(d) 6 UFs, a média aumentou, a mediana aumentou<br>
(e) 6 UFs, a média aumentou, a mediana se manteve<br>

In [ ]:
data = pd.read_csv("./dados/pib_mba_avaliacao.csv")

In [ ]:
data['UF'] = data['UF'].str.upper()

---

### Questão 5)

Desejamos projetar um algoritmo para, por meio dos atributos de entrada 'PIB', 'Pop_ativa_est_2009', 'desemprego', classificar se um município pertence à classes 1/2, ou pertence à classes 3/4/5. Realize os seguintes passos:

1. gere um rótulo/label binário, que é `0` para municípios cujo atributo classe seja 1 e 2, e o rótulo `1` para municípios de classe 3, 4 ou 5;
2. Converta a coluna Pop_ativa_est_2009 para numérico, com opção `downcast='integer', errors='coerce'` para tornar faltante (nan) os não convertidos corretamente.
3. preencha todos os atributos faltantes da base de dados com o valor `-1`;
4. prepare a base de dados de treinamento `X` contendo apenas as colunas 'PIB', 'Pop_ativa_est_2009', 'desemprego', e os rótulos `y` binários conforme passo 1, de forma a ser aceito como entrada nas bibliotecas `sklearn` e `imblearn`;
5. gere uma nova base de dados rebalanceada `Xs` e `ys` por meio do método `under_sampling.RandomUnderSampler(random_state=1)` e usando como estratégia subamostrar a classe majoritária a um tamanho proporcional a 2 vezes o tamanho da classe minoritária;
6. treine o classificador 1 do tipo `DecisionTreeClassifier(max_depth=3, random_state=1)` nos dados originais `X`, `y`, e verifique a acurácia balanceada na predição do conjunto de treinamento `X`, i.e. gerando `y_pred = classif1.predict(X)` e computando `balanced_accuracy_score(y, y_pred)`;
7. treine um novo classificador 2 do tipo `DecisionTreeClassifier(max_depth=3,  random_state=1)` nos dados rebalanceados: `Xs`, `ys`, e verifique a acurácia balanceada na predição no conjunto de treinamento `X`, i.e. gerando `y_pred_s = classif2.predict(X)` e computando `balanced_accuracy_score(y, y_pred_s)`.

OBS: as duas acurácias são computadas no mesmo conjunto `X` relativo ao conjunto de treinamento original.

Em que intervalo recaem as acurácias balanceadas medidas no conjunto **de treinamento** resultantes do classificador 1 (treinado com dados originais) e do classificador 2 (treinado com dados amostrados)?

(a) Classificador 1: [45%, 55%],  Classificador 2: [65%, 72%] <br>
(b) Classificador 1: [45%, 55%],  Classificador 2: [85%, 90%] <br>
(c) Classificador 1: [65%, 72%],  Classificador 2: [65%, 74%] <br>
(d) Classificador 1: [65%, 72%],  Classificador 2: [85%, 90%] <br>
(e) Classificador 1: [76%, 82%],  Classificador 2: [76%, 82%] <br>


In [ ]:
data = pd.read_csv("./dados/pib_mba_avaliacao.csv")

random_state_value = 1